In [157]:
import numpy as np
import matplotlib.pyplot as plt
import skimage as sk
from skimage import io
from skimage import data, color
from skimage.transform import hough_circle
from skimage.feature import peak_local_max, canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte

from glob import glob

%matplotlib inline

In [158]:
def write_dumbbells(filename, accums, centers, radii):
    k = 0
    with open(filename,'w') as f:
        f.write('%d\n' % 2)
        for idx in np.argsort(accums)[::-1][:2]:
            x, y = centers[idx]
            r = radii[idx]
            f.write('%4d %9.4f %9.4f %5d %5d %5d %5d %5d\n' % (k,y,x,r**2,2*r,2*r,r**2*255,-1))
            k+=1

In [159]:
for cam in ('1','2','3','4'):
    db = glob('db/db'+cam+'.?????')
    # print db[0]
    for im in db[:10]:
        image = io.imread(im)
        image_gray = rgb2gray(image)
        # Load picture and detect edges
        image = img_as_ubyte(image_gray)
        edges = canny(image, sigma=3, low_threshold=10, high_threshold=50)

        # fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 8))

        # Detect two radii
        hough_radii = np.arange(30, 60, 2)
        hough_res = hough_circle(edges, hough_radii)

        centers = []
        accums = []
        radii = []

        for radius, h in zip(hough_radii, hough_res):
            # For each radius, extract two circles
            num_peaks = 2
            peaks = peak_local_max(h, num_peaks=num_peaks)
            centers.extend(peaks)
            accums.extend(h[peaks[:, 0], peaks[:, 1]])
            radii.extend([radius] * num_peaks)

        write_dumbbells(im+'_targets',accums,centers,radii)

        # # Draw the most prominent 5 circles
        # image = color.gray2rgb(image)
        # for idx in np.argsort(accums)[::-1][:2]:
        #     center_x, center_y = centers[idx]
        #     radius = radii[idx]
        #     cx, cy = circle_perimeter(center_y, center_x, radius)
        #     image[cy, cx] = (220, 20, 20)

        # ax.imshow(image, cmap=plt.cm.gray)